# [Day 2: Corruption Checksum](https://adventofcode.com/2017/day/2)

## Part A

The first challenge today is very simple - given a "spreadsheet" of numbers, we want to find the difference between the max and min of each row, then output the sum of all these differences. For instance, the output for the spreadsheet

```
5 1 9 5
7 5 3
2 4 6 8
```

should be (9-1) + (7-3) + (8-2) = 18. We assume that we're getting the input as one big string with rows separated by newline characters and the actual numbers separated by tabs. Each row can be treated independently, so first we probably want a helper function that parses the spreadsheet for us into a list of lists.

In [1]:
test_spreadsheet = "5 1 9 5\n7 5 3\n2 4 6 8"

def parse_spreadsheet(spreadsheet):
   return [ [int(x) for x in row.split()] for row in spreadsheet.split("\n") ]

print(parse_spreadsheet(test_spreadsheet))

The above uses a nested list comprehension for brevity (generally I think this should probably be avoided because it can get hard to understand/read, but I think in this case I think it's simple enough to get away with) and takes advantage of Python's `split` function, which splits a string into a list based on the separating string passed in (or whitespace for no argument). Now that we have our spreadsheet in a nice format, we can find the checksum easily by using Python's built in `max` and `min` functions that conveniently already work over lists. List comprehension again lets us do this as a one-liner (and we can even nicely deal with the edge case of empty rows by only looking at the differences of rows of nonzero length).

In [2]:
def test_difference_checksum(f):
    assert(f(test_spreadsheet) == 18)
    assert(f("1 2 3\n4 5 6\n7 8 9") == 6)
    assert(f("2 4 6 8 10\n8 6 4\n2") == 12)
    assert(f("1") == 0)
    assert(f("") == 0)
    assert(f("\n\n\n") == 0)

def find_difference_checksum(spreadsheet):
    rows = parse_spreadsheet(spreadsheet)
    return sum(max(row) - min(row) for row in rows if len(row) > 0)

test_difference_checksum(find_difference_checksum)

## Part B

This one's a bit trickier. We are now told that each row will contain exactly two numbers `a` and `b` such that `a` divides evenly into `b`. Instead of getting the difference between the max and min of each row, we now instead want to find such an `a` and `b` for each row and sum up the values of `b/a`. For instance, if the row was `5 9 2 8` then we want to add 4 to the sum (because `8/2=4` is the only clean division possible between two elements in the row). So, the problem really comes down to figuring out how to find out which two numbers in the row divide properly. The simplest way to do so (and also the only one I've been able to come up with...) is to just iterate through each possible pair and check divisibility. 

In [3]:
def test_divisible_checksum(f):
    assert(f("2 3 4") == 2)
    assert(f("5 9 2 8\n9 4 7 3\n3 8 6 5") == 9)
    assert(f("1") == 0)
    assert(f("") == 0)

def find_divisible_checksum(spreadsheet):
    rows = parse_spreadsheet(spreadsheet)
    
    # We assume that a row without a divisible pair counts as 0
    # for the sum (it'd also make sense to throw an exception or 
    # something since that shouldn't happen).
    def get_divisible_in_row(row):
        N = len(row)
        for i in range(N):
            for j in range(N):
                if i != j and row[i] % row[j] == 0:
                    return row[i] // row[j]
        return 0

    return sum(get_divisible_in_row(row) for row in rows)

test_divisible_checksum(find_divisible_checksum)

Note that theoretically we also could've implemented get_divisible_in_row as a big list comprehension:
```
[a // b for (a, i) in enumerate(row) for (b, j) in enumerate(row) if i != j and a % b == 0 ]
```
and actually even the entire function (by adding a third nested for over rows) but that seemed a bit overly convoluted to me. Also, having `get_divisible_in_row` as a separate helper function lets us break out of the nested fors early since we're told there's only a single divisible pair in the row.

Now let's see if we can speed up our solution. Note that we're currently iterating all the *permutations* (so we'll be looking at both `(list[i], list[j])` and `(list[j], list[i])`; these represent the same *combination* of indices but different permutations). If we were to use combinations, we would get to skip half the checks and our for loop might look something like

```python
for i in range(N):
    for j in range(i+1,N):
        ...
```

However, order matters in our case because even if `a` doesn't divide `b`, it could be the case that `b` divides `a`. Thankfully, we can get around this by noting that a smaller number can never be divided into cleanly by a larger number. In other words, we only care about the pairs `(a,b)` where `b > a` and can ignore `(b,a)` altogether.

To ensure this, all we need to do is sort the list we're passing in first. Then if we iterate through the combinations such that in `(i,j)`, `i < j` (which is what we do in teh above for loop), we'll know that `(j,i)` can be ignored.

Finally, to clean up our code a bit, rather than writing the combination generation with nested `for`s as we've been doing we can use Python's  `itertools` library, which is generally helpful for doing anything vaguely combinatoric. In particular, `itertools.combinations(list, 2)` will return all the combinations of two distinct elements of a list. Note that `itertools.combniations(list, 3)` would give you the triplets, `itertools.combinations(list, 4)` would give you the 4-tuples, etc. . As an example:

In [4]:
import itertools

pairs = itertools.combinations([1,2,2,3], 2)
for pair in pairs:
    print(pair)

(1, 2)
(1, 2)
(1, 3)
(2, 2)
(2, 3)
(2, 3)


i.e finding the possible combinations of the list (based on position rather than value).

`itertools.combinations` also has the benefit of being slightly faster than using the regular for loop construct. Now we can simplify our code to the following (note we could've done something similar with `itertools.permutations` in our original method):

In [5]:
def find_divisible_checksum_with_combinations(spreadsheet):
    rows = parse_spreadsheet(spreadsheet)
    
    def get_divisible_in_row(row):
        for (a, b) in itertools.combinations(sorted(row), 2):
            if b % a == 0:
                return b // a
        return 0

    return sum(get_divisible_in_row(row) for row in rows)

test_divisible_checksum(find_divisible_checksum_with_combinations)

Some final notes on timing. Using itertools.combinations like this (or more generally using the `for i in range(N), for j in range(i+1,N)` construct) halves the number of pairs we need to look at for each row, BUT comes at the cost of needing to sort the row. Note however that sorting can be done in `O(N lg N)` while the number of pairs we have to look at is `O(N^2)`, so (at least asymptotically) getting to halve the number of pairs should be well worth the additional sorting for large `N`.

## Acknowledgements

Thanks to [keyao21](https://github.com/keyao21) for pointing out a mistake in my timing analysis.